In [ ]:
import os
os.chdir("/content")
!rm -rf "AVSS"
!ls

sample_data


In [ ]:
!git clone https://github.com/ArthurGaleev/AVSS.git

Cloning into 'AVSS'...
remote: Enumerating objects: 1681, done.
remote: Counting objects: 100% (416/416), done.
remote: Compressing objects: 100% (277/277), done.
remote: Total 1681 (delta 233), reused 245 (delta 129), pack-reused 1265 (from 1)
Receiving objects: 100% (1681/1681), 337.63 KiB | 1.83 MiB/s, done.
Resolving deltas: 100% (1117/1117), done.


In [6]:
import os
os.chdir("AVSS")

In [ ]:
!pip install uv
!uv sync

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 125.2 MB/s eta 0:00:00
Using CPython 3.13.7
Creating virtual environment at: .venv
Resolved 145 packages in 3.38s
Prepared 142 packages in 1m 20s
Installed 142 packages in 1.00s
 + annotated-types==0.7.0
 + antlr4-python3-runtime==4.9.3
 + attrs==25.4.0
 + audioop-lts==0.2.2
 + audioread==3.1.0
 + beautifulsoup4==4.14.2
 + black==25.9.0
 + bleach==6.3.0
 + certifi==2025.11.12
 + cffi==2.0.0
 + cfgv==3.5.0
 + charset-normalizer==3.4.4
 + click==8.3.1
 + coloredlogs==15.0.1
 + comet-ml==3.54.0
 + configobj==5.0.9
 + contourpy==1.3.3
 + cycler==0.12.1
 + decorator==5.2.1
 + distlib==0.4.0
 + dotenv==0.9.9
 + dulwich==0.24.10
 + everett==3.1.0
 + filelock==3.20.0
 + flatbuffers==25.9.23
 + fonttools==4.60.1
 + fsspec==2025.10.0
 + gammatone==1.0.3
 + gdown==5.2.0
 + gitdb==4.0.12
 + gitpython==3.1.45
 + humanfriendly==10.0
 + hydra-core==1.3.2
 + identify==2.6.15
 + idna==3.11
 + isort==7.0.0
 + jinja2==3.1.6
 + joblib==1.5.2
 + json

In [ ]:
# download our dataset and all models.
# !uv run scripts/download_gdrive.py

In [7]:
import sys
sys.path.append("/content/AVSS")

In [9]:
from scripts.download_gdrive import download_dataset, download_models
RTFS_MODEL_PATH="data/models/rtfs-4-reuse.pth" #Our SOTA Model
DPRNN_MODEL_PATH="data/models/dprnn.pth" #Our best dprnn checkpoint
DATASET_PATH="data/datasets"#Where you download dataset from google drive
DATASET_LINK="https://drive.google.com/uc?id=1wu_S9wpiZmpyAZosvWTCYC9W6zD6nAVe"
#If returns
GDRIVE_URLS = {
    "models": {
        # rtfs-4-reuse, links can be found in scripts/download_gdrive
        "https://drive.google.com/uc?id=1lzO_cyN0WKof1rtnCPZpN-BHcELO-85c": RTFS_MODEL_PATH,
        "https://drive.google.com/uc?id=1IU4mFseGPtac6g58jTS48GzlMI1wMJQ8":DPRNN_MODEL_PATH
    },
    "dataset": {
        DATASET_LINK:DATASET_PATH#Put your dataset link here
    }
}
download_models(GDRIVE_URLS) #if models dont download you can download them manually to data/models
download_dataset(GDRIVE_URLS) #if this doesnt work run cell below

In [ ]:
#download dataset from https://drive.google.com/file/d/1wu_S9wpiZmpyAZosvWTCYC9W6zD6nAVe/view?usp=sharing to your google drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# !find /content/drive/ -type f -name "*.zip"

In [ ]:
# !unzip /content/drive/MyDrive/dla_dataset_small_av.zip -d /content/AVSS/data/datasets

In [ ]:
#you can also specify a link to your yandex disk zip dataset for it to download during training ot infernce, see src/datasets/yandex_download for details
# os.environ["YANDEX_DISK_URL"]="https://disk.360.yandex.ru/d/5pz96ysIZi33IQ"

In [17]:
!uv run inference.py batch_size=1 inferencer.save_path="pred_dprnn_small_av" use_pit=True   inferencer.from_pretrained={DPRNN_MODEL_PATH} download_name=dla_dataset_small_av model=dprnn -cn=inference

Using GPU: cuda:0 with 14.64 GB free
Creating dataset: 100% 500/500 [00:08<00:00, 58.37it/s]
Creating dataset: 100% 3000/3000 [00:16<00:00, 183.80it/s]
DPRNN_TasNet(
  (encoder): Encoder(
    (dconv1d_block): Sequential(
      (0): Conv1d(1, 64, kernel_size=(1,), stride=(1,))
      (1): PReLU(num_parameters=1)
      (2): GroupNorm(1, 64, eps=1e-05, affine=True)
      (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), groups=64)
      (4): PReLU(num_parameters=1)
      (5): GroupNorm(1, 64, eps=1e-05, affine=True)
    )
    (res_conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (out_conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  )
  (sep_start): Sequential(
    (0): GroupNorm(1, 64, eps=1e-05, affine=True)
    (1): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  )
  (separation): DPRNN(
    (segmentation): Segmentation()
    (dprnn_blocks): ModuleList(
      (0-5): 6 x DPRNNBlock(
        (intra_rnn): LSTM(64, 128, batch_first=True, bidirectional=True)
    

In [11]:
!uv run inference.py inferencer.save_path="pred_rtfs_small_av" \
 inferencer.from_pretrained={RTFS_MODEL_PATH}  download_name=dla_dataset_small_av model=rtfs-4-reuse  -cn=inference

Using GPU: cuda:0 with 14.64 GB free
Creating dataset: 100% 500/500 [00:07<00:00, 67.96it/s]
Creating dataset: 100% 3000/3000 [00:17<00:00, 170.33it/s]
RTFSModel(
  (stft): TransformSTFT()
  (audio_encoder): RTFSAudioEncoder(
    (conv): Conv2d(2, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (ap_block): RTFSBlock(
    (compressor): CompressorBlock(
      (dimension_compression_layer): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (tf_compression_layers): Sequential(
        (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
        (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
      )
    )
    (reconstructor): ReconstructionBlock(
      (g_fuse_layers): ModuleList(
        (0-2): 3 x TFARUnit(
          (w1_pathway): Sequential(
            (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1), padding=same, groups=128)
            (1): GlobalLayerNorm(1, 128, eps=1e-08, affine=True

In [12]:
#calculate metrics on predictions on the dla_dataset_small_av folder of rtfs model
!uv run calc_metrics.py --gt_s1=data/datasets/dla_dataset_small_av/audio/val/s1 --gt_s2=data/datasets/dla_dataset_small_av/audio/val/s2 \
    --pred_s1=data/saved/pred_rtfs_small_av/val/s1 --pred_s2=data/saved/pred_rtfs_small_av/val/s2  --mix=data/datasets/dla_dataset_small_av/audio/val/mix

Calculating Metrics...: 100% 500/500 [01:58<00:00,  4.24it/s]
Average Metrics:
STOI: 0.8330
PESQ: 1.9828
SI-SDR-I: 8.4503
SI-SNR-I: 8.4524


In [14]:
import time
start = time.time()

!uv run inference.py inferencer.save_path="" \
    batch_size=1 metrics.inference=[]\
    inferencer.from_pretrained={RTFS_MODEL_PATH} \
    download_name=dla_dataset_small_av\
    model=rtfs-4-reuse inferencer.device=cpu  datasets.test=null \
    +datasets.val.limit=1 -cn=inference

end = time.time()

Creating dataset: 100% 500/500 [00:07<00:00, 65.81it/s]
RTFSModel(
  (stft): TransformSTFT()
  (audio_encoder): RTFSAudioEncoder(
    (conv): Conv2d(2, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (ap_block): RTFSBlock(
    (compressor): CompressorBlock(
      (dimension_compression_layer): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
      (tf_compression_layers): Sequential(
        (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
        (1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), groups=128)
      )
    )
    (reconstructor): ReconstructionBlock(
      (g_fuse_layers): ModuleList(
        (0-2): 3 x TFARUnit(
          (w1_pathway): Sequential(
            (0): Conv2d(128, 128, kernel_size=(4, 4), stride=(1, 1), padding=same, groups=128)
            (1): GlobalLayerNorm(1, 128, eps=1e-08, affine=True)
            (2): Sigmoid()
          )
          (w2_pathway): Sequential(
            (0): Co

In [16]:
print("Total elapsed:", end - start, "seconds")
print("Per batch Time: ", 9, " seconds") #can be seen on the tqdm bar: val: 100% 1/1 [00:09<00:00,  9.09s/it]
#you can increase batch_size and divide by batch_size to see time the implementation

Total elapsed: 23.01067066192627 seconds
Per batch Time:  9  seconds
